# TerrainSandbox 

**Author**: Chris Sheehan

**Block 1: The Grid**

This block creates a rectangular grid that will serve as the model domain. If you have ever worked with a digital elevation model (DEM) or any other kind of raster file, then this might be familiar to you. The grid will consist of a series of nodes (which are similar to the cells in a raster...it’s a bit more complicated than that, but this is a useful way to think about it!), and the nodes can store different kinds of values (such as topographic elevation...i.e. just like a DEM). 

You can control the dimensions and spatial resolution of the grid:

* **number_of_rows**: Number of rows in the grid 

* **number_of_columns**: Number of columns in the grid

* **dxy**: Spatial resolution of the grid (i.e. the spacing between each grid node...you can think of this as the length of each cell in a raster).

In [ ]:
# BLOCK 1: CREATE THE GRID

#############################################################################################################################################################
#############################################################################################################################################################

number_of_rows = 200            
number_of_columns = 200       
dxy = 100                      

#############################################################################################################################################################
#############################################################################################################################################################

import numpy as np
from landlab import RasterModelGrid, imshow_grid
from landlab.components import StreamPowerEroder, LinearDiffuser, FlowAccumulator, ChannelProfiler, SteepnessFinder, ChiFinder
from matplotlib import pyplot as plt
import os
import os.path
from os import path
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore")
mg = RasterModelGrid((number_of_rows, number_of_columns), dxy)
zr = mg.add_zeros('node', 'topographic__elevation')
erosion_rate = mg.add_zeros('node', 'erosion_rate')
previous_zr = mg.add_zeros('node', 'previous_topographic__elevation')
dz_dt = mg.add_zeros('node', 'elevational_change')
np.random.seed(0)                                      
mg_noise = np.random.rand(mg.number_of_nodes)/1000.    
zr += mg_noise

In [ ]:
# BLOCK 2: SET BOUNDARY CONDITIONS

#############################################################################################################################################################
#############################################################################################################################################################

East = 1
North = 1
West = 1
South = 1

#############################################################################################################################################################
#############################################################################################################################################################

mg.set_status_at_node_on_edges(right=East, top=North, left=West, bottom=South)

In [ ]:
# BLOCK 3: LITHOLOGIC PARAMETERS

#############################################################################################################################################################
#############################################################################################################################################################

Ksp_1 = 5E-4
Khs_1 = 1E-3

Ksp_2 = 0
Khs_2 = 0
K_Edge_2 = 0

Ksp_3 = 0
Khs_3 = 0
K_Edge_3 = 0

Ksp_4 = 0
Khs_4 = 0
K_Edge_4 = 0

Ksp_5 = 0
Khs_5 = 0
K_Edge_5 = 0

m_sp = 0.5
n_sp = 1

#############################################################################################################################################################
#############################################################################################################################################################

Ksp = np.ones(mg.number_of_nodes) * Ksp_1 
Khs = np.ones(mg.number_of_nodes) * Khs_1   
if K_Edge_2 > 0:
    Ksp[np.where(mg.node_x>K_Edge_2)] = Ksp_2 
    Khs[np.where(mg.node_x>K_Edge_2)] = Khs_2
if K_Edge_3 > 0:
    Ksp[np.where(mg.node_x>K_Edge_3)] = Ksp_3 
    Khs[np.where(mg.node_x>K_Edge_3)] = Khs_3
if K_Edge_4 > 0:
    Ksp[np.where(mg.node_x>K_Edge_4)] = Ksp_4 
    Khs[np.where(mg.node_x>K_Edge_4)] = Khs_4
if K_Edge_5 > 0:
    Ksp[np.where(mg.node_x>K_Edge_5)] = Ksp_5 
    Khs[np.where(mg.node_x>K_Edge_5)] = Khs_5 

In [ ]:
# BLOCK 4: TECTONIC PARAMETERS

#############################################################################################################################################################
#############################################################################################################################################################

U_1 = 1E-3

U_2 = 2E-3
U_Edge_2 = 4000

U_3 = 3E-3
U_Edge_3 = 8000

U_4 = 4E-3
U_Edge_4 = 12000

U_5 = 5E-3
U_Edge_5 = 16000

#############################################################################################################################################################
#############################################################################################################################################################

U = np.ones(mg.number_of_nodes) * U_1 
if U_Edge_2 > 0:
    U[np.where(mg.node_x>U_Edge_2)] = U_2 
if U_Edge_3 > 0:
    U[np.where(mg.node_x>U_Edge_3)] = U_3 
if U_Edge_4 > 0:
    U[np.where(mg.node_x>U_Edge_4)] = U_4 
if U_Edge_5 > 0:
    U[np.where(mg.node_x>U_Edge_5)] = U_5 

In [ ]:
# BLOCK 5: RESET THE MODEL TIME AND THE PLOTTING TIMERS TO 0

total_time = 0 
Plot_Ticker = 0
Export_DEM_Ticker = 0

In [ ]:
# BLOCK 6: SET THE TIMESTEP AND THE MODEL RUN-TIME

#############################################################################################################################################################
#############################################################################################################################################################

dt = 100          
tmax = 1E5        

#############################################################################################################################################################
#############################################################################################################################################################

t = np.arange(0, tmax, dt) 

In [ ]:
# BLOCK 7: PLOTTING OPTIONS

#############################################################################################################################################################
#############################################################################################################################################################

Plot_interval = 2E4
Export_DEM_Interval = 1000
number_of_watersheds = 1
min_drainage_area = 1000000
main_channel_only = True

Directory = '/Users/Chris/Documents/'
DEM_Image = True
Slope_Area = True
Channel_Profile = True
Channel_Map = True
Ksn_Profile = True
Ksn_Map = True
Chi_Profile = True
Chi_Map = True
Erosion_Rate_Profile = True
Erosion_Rate_Map = True
DZ_DT_Profile = True
DZ_DT_Map = True
Ksp_Profile = True
Ksp_Map = True

#############################################################################################################################################################
#############################################################################################################################################################

In [ ]:
# BLOCK 8: "THE MODEL"

frr = FlowAccumulator(mg, flow_director='D8') 
spr = StreamPowerEroder(mg, K_sp=Ksp, m_sp=m_sp, n_sp=n_sp)
dfn = LinearDiffuser(mg, linear_diffusivity=Khs)
previous_zr[mg.core_nodes] = zr[mg.core_nodes]
for ti in t:
    zr[mg.core_nodes] += U[mg.core_nodes]*dt            
    dfn.run_one_step(dt)                                
    frr.run_one_step()                                  
    spr.run_one_step(dt)
    dz_dt[mg.core_nodes] = (zr[mg.core_nodes] - previous_zr[mg.core_nodes]) / dt
    erosion_rate[mg.core_nodes] = (previous_zr[mg.core_nodes] - (zr[mg.core_nodes] - (U[mg.core_nodes]*dt))) / dt
    previous_zr[mg.core_nodes] = zr[mg.core_nodes]
    total_time += dt                                    
    print(total_time)
    Plot_Ticker += dt
    Export_DEM_Ticker += dt
    if Plot_Ticker == Plot_interval:
        if main_channel_only == True:
            prf = ChannelProfiler(mg, number_of_watersheds=number_of_watersheds, main_channel_only=True, minimum_channel_threshold=min_drainage_area)
        if main_channel_only == False:
            prf = ChannelProfiler(mg, number_of_watersheds=number_of_watersheds, main_channel_only=False, minimum_channel_threshold=min_drainage_area)
        if total_time == Plot_Ticker:
            sf = SteepnessFinder(mg, reference_concavity=m_sp/n_sp, min_drainage_area=min_drainage_area)
            cf = ChiFinder(mg, reference_concavity=m_sp/n_sp, min_drainage_area=min_drainage_area)
            if path.exists(str(Directory)+'/TerrainSandbox') == False:
                os.mkdir(str(Directory)+'/TerrainSandbox')
        if DEM_Image == True:
            if path.exists(str(Directory)+'/TerrainSandbox/DEM_Image') == False:
                os.mkdir(str(Directory)+'/TerrainSandbox/DEM_Image')
            plt.ioff()
            fig = plt.figure(1)         
            imshow_grid(mg, 'topographic__elevation', grid_units=('m', 'm'), var_name="Elevation (m)", cmap='terrain', allow_colorbar=True)
            title_text = '$Year$='+str(total_time)  
            plt.title(title_text)
            plt.tight_layout()
            fig.savefig(str(Directory)+'/TerrainSandbox/DEM_Image/'+str(total_time)+'.pdf',  format='pdf', dpi=300)
            plt.close(fig)
        if Slope_Area == True:
            prf.run_one_step()
            if path.exists(str(Directory)+'/TerrainSandbox/Slope_Area') == False:
                os.mkdir(str(Directory)+'/TerrainSandbox/Slope_Area')
            plt.ioff()
            fig = plt.figure(2)
            for i, outlet_id in enumerate(prf.data_structure):
                for j, segment_id in enumerate(prf.data_structure[outlet_id]):
                    if j == 0:
                        label = "channel {i}".format(i=i + 1)
                    else:
                        label = '_nolegend_'
                    segment = prf.data_structure[outlet_id][segment_id]
                    profile_ids = segment["ids"]
                    color = segment["color"]
                    plt.loglog(mg.at_node["drainage_area"][profile_ids], mg.at_node["topographic__steepest_slope"][profile_ids], '.', color=color, label=label)
            plt.legend(loc="lower left")
            plt.xlabel("Drainage Area (m^2)")
            plt.ylabel("Channel Slope [m/m]")
            title_text = '$Year$='+str(total_time)  
            plt.title(title_text)
            plt.grid(linestyle='--')
            plt.tight_layout()
            fig.savefig(str(Directory)+'/TerrainSandbox/Slope_Area/'+str(total_time)+'.pdf',  format='pdf', dpi=300)
            plt.close(fig)
        if Channel_Profile == True:
            prf.run_one_step()
            if path.exists(str(Directory)+'/TerrainSandbox/Channel_Profile') == False:
                os.mkdir(str(Directory)+'/TerrainSandbox/Channel_Profile')
            plt.ioff()
            fig = plt.figure(3)
            prf.plot_profiles(field='topographic__elevation', xlabel='Distance Upstream (m)', ylabel='Elevation (m)')
            title_text = '$Year$='+str(total_time)
            plt.title(title_text)
            plt.grid(linestyle='--')
            plt.tight_layout()
            fig.savefig(str(Directory)+'/TerrainSandbox/Channel_Profile/'+str(total_time)+'.pdf',  format='pdf', dpi=300)
            plt.close(fig)
        if Channel_Map == True:
            prf.run_one_step()
            if path.exists(str(Directory)+'/TerrainSandbox/Channel_Map') == False:
                os.mkdir(str(Directory)+'/TerrainSandbox/Channel_Map')
            plt.ioff()
            fig = plt.figure(4)
            prf.plot_profiles_in_map_view()
            title_text = '$Year$='+str(total_time)
            plt.title(title_text)
            plt.tight_layout()
            fig.savefig(str(Directory)+'/TerrainSandbox/Channel_Map/'+str(total_time)+'.pdf',  format='pdf', dpi=300)
            plt.close(fig)
        if Ksn_Profile == True:
            prf.run_one_step()
            sf.calculate_steepnesses()
            if path.exists(str(Directory)+'/TerrainSandbox/Ksn_Profile') == False:
                os.mkdir(str(Directory)+'/TerrainSandbox/Ksn_Profile')
            plt.ioff()
            fig = plt.figure(5)
            for i, outlet_id in enumerate(prf.data_structure):
                for j, segment_id in enumerate(prf.data_structure[outlet_id]):
                    if j == 0:
                        label = "channel {i}".format(i=i + 1)
                    else:
                        label = '_nolegend_'
                    segment = prf.data_structure[outlet_id][segment_id]
                    profile_ids = segment["ids"]
                    distance_upstream = segment["distances"]
                    color = segment["color"]
                    plt.plot(distance_upstream, mg.at_node["channel__steepness_index"][profile_ids], 'x', color=color, label=label)
            plt.xlabel("Distance Upstream (m)")
            plt.ylabel("Steepness Index")
            plt.legend(loc="upper left")
            title_text = '$Year$='+str(total_time)  
            plt.title(title_text)
            plt.grid(linestyle='--')
            plt.tight_layout()
            fig.savefig(str(Directory)+'/TerrainSandbox/Ksn_Profile/'+str(total_time)+'.pdf',  format='pdf', dpi=300)
            plt.close(fig)
        if Ksn_Map == True:
            prf.run_one_step()
            sf.calculate_steepnesses()
            if path.exists(str(Directory)+'/TerrainSandbox/Ksn_Map') == False:
                os.mkdir(str(Directory)+'/TerrainSandbox/Ksn_Map')
            plt.ioff()
            fig = plt.figure(6)
            imshow_grid(mg, "channel__steepness_index", grid_units=("m", "m"), var_name="Steepness Index", cmap="jet")
            title_text = '$Year$='+str(total_time)  
            plt.title(title_text)
            plt.tight_layout()
            fig.savefig(str(Directory)+'/TerrainSandbox/Ksn_Map/'+str(total_time)+'.pdf',  format='pdf', dpi=300)
            plt.close(fig)
        if Chi_Profile == True:
            prf.run_one_step()
            cf.calculate_chi()
            if path.exists(str(Directory)+'/TerrainSandbox/Chi_Profile') == False:
                os.mkdir(str(Directory)+'/TerrainSandbox/Chi_Profile')
            plt.ioff()
            fig = plt.figure(7)
            for i, outlet_id in enumerate(prf.data_structure):
                for j, segment_id in enumerate(prf.data_structure[outlet_id]):
                    if j == 0:
                        label = "channel {i}".format(i=i + 1)
                    else:
                        label = '_nolegend_'
                    segment = prf.data_structure[outlet_id][segment_id]
                    profile_ids = segment["ids"]
                    color = segment["color"]
                    plt.plot(mg.at_node["channel__chi_index"][profile_ids], mg.at_node["topographic__elevation"][profile_ids], color=color, label=label)
            plt.xlabel("Chi Index (m)")
            plt.ylabel("Elevation (m)")
            plt.legend(loc="lower right")
            title_text = '$Year$='+str(total_time)  
            plt.title(title_text)
            plt.grid(linestyle='--')
            plt.tight_layout()
            fig.savefig(str(Directory)+'/TerrainSandbox/Chi_Profile/'+str(total_time)+'.pdf',  format='pdf', dpi=300)
            plt.close(fig)
        if Chi_Map == True:
            prf.run_one_step()
            cf.calculate_chi()
            if path.exists(str(Directory)+'/TerrainSandbox/Chi_Map') == False:
                os.mkdir(str(Directory)+'/TerrainSandbox/Chi_Map')
            plt.ioff()
            fig = plt.figure(8)
            imshow_grid(mg, "channel__chi_index", grid_units=("m", "m"), var_name="Chi Index", cmap="jet")
            title_text = '$Year$='+str(total_time)  
            plt.title(title_text)
            plt.tight_layout()
            fig.savefig(str(Directory)+'/TerrainSandbox/Chi_Map/'+str(total_time)+'.pdf',  format='pdf', dpi=300)
            plt.close(fig)
        if Erosion_Rate_Profile == True:
            prf.run_one_step()
            if path.exists(str(Directory)+'/TerrainSandbox/Erosion_Rate_Profile') == False:
                os.mkdir(str(Directory)+'/TerrainSandbox/Erosion_Rate_Profile')
            plt.ioff()
            fig = plt.figure(3)
            prf.plot_profiles(field='erosion_rate', xlabel='Distance Upstream (m)', ylabel='Erosion Rate (m/yr)')
            title_text = '$Year$='+str(total_time)
            plt.title(title_text)
            plt.grid(linestyle='--')
            axes = plt.gca()
            axes.set_xlim([dxy,None])
            plt.tight_layout()
            fig.savefig(str(Directory)+'/TerrainSandbox/Erosion_Rate_Profile/'+str(total_time)+'.pdf',  format='pdf', dpi=300)
            plt.close(fig)
        if Erosion_Rate_Map == True:
            prf.run_one_step()
            if path.exists(str(Directory)+'/TerrainSandbox/Erosion_Rate_Map') == False:
                os.mkdir(str(Directory)+'/TerrainSandbox/Erosion_Rate_Map')
            plt.ioff()
            fig = plt.figure(8)
            imshow_grid(mg, "erosion_rate", grid_units=("m", "m"), var_name="Erosion Rate (m/yr)", cmap="jet")
            title_text = '$Year$='+str(total_time)  
            plt.title(title_text)
            plt.tight_layout()
            fig.savefig(str(Directory)+'/TerrainSandbox/Erosion_Rate_Map/'+str(total_time)+'.pdf',  format='pdf', dpi=300)
            plt.close(fig)
        if DZ_DT_Profile == True:
            prf.run_one_step()
            if path.exists(str(Directory)+'/TerrainSandbox/DZDT_Profile') == False:
                os.mkdir(str(Directory)+'/TerrainSandbox/DZDT_Profile')
            plt.ioff()
            fig = plt.figure(3)
            prf.plot_profiles(field='elevational_change', xlabel='Distance Upstream (m)', ylabel='Rate of Elevational Change (m/yr)')
            title_text = '$Year$='+str(total_time)
            plt.title(title_text)
            plt.grid(linestyle='--')
            axes = plt.gca()
            axes.set_xlim([dxy,None])
            plt.tight_layout()
            fig.savefig(str(Directory)+'/TerrainSandbox/DZDT_Profile/'+str(total_time)+'.pdf',  format='pdf', dpi=300)
            plt.close(fig)
        if DZ_DT_Map == True:
            prf.run_one_step()
            if path.exists(str(Directory)+'/TerrainSandbox/DZDT_Map') == False:
                os.mkdir(str(Directory)+'/TerrainSandbox/DZDT_Map')
            plt.ioff()
            fig = plt.figure(8)
            imshow_grid(mg, "elevational_change", grid_units=("m", "m"), var_name="Rate of Elevational Change (m/yr)", cmap="jet")
            title_text = '$Year$='+str(total_time)  
            plt.title(title_text)
            plt.tight_layout()
            fig.savefig(str(Directory)+'/TerrainSandbox/DZDT_Map/'+str(total_time)+'.pdf',  format='pdf', dpi=300)
            plt.close(fig)
        if Ksp_Profile == True:
            prf.run_one_step()
            if path.exists(str(Directory)+'/TerrainSandbox/Ksp_Profile') == False:
                os.mkdir(str(Directory)+'/TerrainSandbox/Ksp_Profile')
            plt.ioff()
            fig = plt.figure(117)
            prf.plot_profiles(field=Ksp, xlabel='Distance Upstream (m)', ylabel='Ksp')
            title_text = '$Year$='+str(total_time) 
            plt.title(title_text)
            plt.grid(linestyle='--')
            plt.tight_layout()
            fig.savefig(str(Directory)+'/TerrainSandbox/Ksp_Profile/'+str(total_time)+'.pdf',  format='pdf', dpi=300)
            plt.close(fig)
        if Ksp_Map == True:
            prf.run_one_step()
            if path.exists(str(Directory)+'/TerrainSandbox/Ksp_Map') == False:
                os.mkdir(str(Directory)+'/TerrainSandbox/Ksp_Map')
            plt.ioff()
            fig = plt.figure(8)
            imshow_grid(mg, Ksp, grid_units=("m", "m"), var_name="Ksp", cmap="jet")
            title_text = '$Year$='+str(total_time)  
            plt.title(title_text)
            plt.tight_layout()
            fig.savefig(str(Directory)+'/TerrainSandbox/Ksp_Map/'+str(total_time)+'.pdf',  format='pdf', dpi=300)
            plt.close(fig)
        Plot_Ticker = 0